

> # **Advanced Image Processing**



# Blob Detection
## What are blobs and why do we care?
In essence, "blob detection" is the process of searching for and identifying bright spots on a dark background, or dark spots on a light background. The "blobs" can correspond to any number of physical phenomena. In astronomy, for example, this can be relevant if you have an image of a part of the sky and want to identify something like galaxies. You might be wondering though, why not just look at an image yourself and pick out blobs "by eye"? In our example of identifying galaxies in an astronomical image, a systematic way of doing this may be beneficial for several reasons: 
- you may have faint objects in your image the are difficult to distinguish from the background
- you may have a crowded field where bright points are close enough together that they are difficult to disentangle
- you may have a large dataset where it would just take you a long time to go through everything by hand
- you may want to define where a blob "ends" in a systematic way so that your blob definition is consistent across all objects you identify

These are just some of the reasons why a systematic approach to identifying blobs in images could be beneficial. Checking the output of your algorithm by eye, however, is wise to make sure that it is not outputting nonsense.

Let's get started with first reading in our first astronomical image; a nearby group of galaxies! Astronomical images are stored as "fits" files, this is essentially a table that contains the information of how bright the sky is at each pixel in your image, and a "header", which contains information about how pixels translate into coordinates on the sky, what instrument was used, and the units of the image for example. We will read the fits file into a numpy array using the package called astropy (see http://docs.astropy.org/en/stable/index.html for documentation). This package streamlines the process of working with astronomical images in Python. 

First we will import a few packages that we are going to use throughout this section: skimage will allow us to run the blob detection algorithms, matplotlib  will allow us to plot our data, astropy will allow us to read "fits" files,  and numpy will let us do cool things with arrays.

In [ ]:
from skimage.feature import blob_dog, blob_log, blob_doh
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np



This line reads the fits file --> data table and header.

In [ ]:
hdu = fits.open('./RSCG1.fits')

# Exercise 5: hdu

What is the type of hdu? What happens when you print hdu?

In [ ]:
#Insert your code here:

print(type(hdu))
print(hdu)

This line reads the data associated with the header into a numpy array. A fits file can have more than one header, and since Python indices start at 0, we tell Python to read the data associated with the first (and only) header associated with this fits file. 

In [ ]:
image = hdu[0].data
# This line closes the fits file because we don't need it anymore; we've already read the data into an array.
hdu.close()

# Exercise 6: Astronomical Images
What is the type of the image? What happens when you print image? What are the dimensions of image? What are the minimum and maximum values?

In [ ]:
#Insert your code here:

print(type(image))
print(image)
print(image.shape)
print(np.max(image))
print(np.min(image))


That's it, you've now read your astronomical image into Python and can work with the data and analyze it! Let's visualize the data now to get an idea of what it is we are actually dealing with.

In [ ]:
# imshow will map the 2D array (our data), and origin='lower' tells imshow to plot (0,0) in the bottom left corner. 
plt.imshow(image,origin='lower')
plt.colorbar()
plt.show()


# EXERCISE: What happens if you remove origin='lower'? How does the mapping of the image change?

We can now run some blob finding algorithms on this data set and see what we find and if it makes sense! For this, we will use the scikit-image package which is an image processing pacakge in Python. This package has three algorithms for detecting blobs (all discussed here: https://en.wikipedia.org/wiki/Blob_detection):
- Laplacian of Gaussian
- Difference of Gaussian
- Determinant of Hessian

This is great, but what does this actually mean? How do they work? What's the difference between them?

### Laplacian of Gaussian
This algorithm starts by smoothing (blurring) the entire image with a two-dimensional Gaussian function: 

$$\begin{align} g(x,y,\sigma) = \frac{1}{2\pi\sigma^2} e^\left(-\frac{x^2+y^2}{2\sigma^2}\right) \end{align}$$ 

where $(x,y)$ correspond to pixel coordinates on your image, and $\sigma$ represents the "size" or "width" of the Gaussian function. The algorithm then performs a mathematical computation called taking the "Laplacian", however we will not go into the details of that. What this effectively gives you is strong responses for blobs of size $\sqrt{2}\sigma$. This means that the algorithm is most sensitive to detecting blobs that have a similar size to that of the Gaussian function that you smooth your image with in the first place. In order to detect blobs of varying size, you need to use an approach that allows you to smooth your image with Gaussians of varying sizes to try to match the scale of potential blobs. This approach is the most accurate but also the slowest approach, especially for the largest blobs, because it needs to smooth the image with a larger Gaussian.

The scikit-image function "blob_log" (https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log) has this capability. To run it, we simply need to call the function and provide it with the correct input parameters. In the following piece of code, this is what we are going to provide the function with:
- **image** to run the blob detection on
- **min_sigma**: the minimum size for the Gaussian that will be used to smooth the image; this corresponds to the smallest blobs we expect in the image
- **max_sigma**: the maximum size for the Gaussian that will be used to smooth the image; this corresponds to the largest blobs we expect in the image
- **num_sigma**: the number of sizes to consider between the minimum and maximum
- **threshold**: relates to how faint of a blob you can detect; a smaller threshold will detect fainter blobs

The call to this function will output a list of x and y coordinates corresponding to the center of each blob, as well as the "size" of the Gaussian that it used to find that blob. Let's try it out!

In [ ]:
# This line calls the blob detection (Laplacian of Gaussian function) on our galaxies image with the parameters that
# we provide
blobs_log = blob_log(image, min_sigma = 2, max_sigma=9, num_sigma=8, threshold=.05)

# Exercise 7: Blobs 
What kind of output does blobs_log provide? What are its dimensions? What does the length of blobs_log physically correspond to?

In [ ]:
input("What kind of output is blobs_log?")
input("What are its dimension?")
input("What does the length of blobs_log physically correspond to?")

## Data from the blob finder 

The first column of blobs_log (blobs_log[0]) is the y-coordinate, the second column (blobs_log[1]) is the x-coordinate, and the third column (blobs_log[2]) is the "size" of the Gaussian that was used to detect the blob. 

In [ ]:
# In this line we use the third column to calculate the radius of the blob itself.
blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)

## Plotting the blobs
Now we'll plot the blobs onto our image! For this, we will create circles with plt.Circle using the corrdinate and radius information we have from blobs_log. Then we will use add_patch to add that circle to our image. Since the plotting software needs information about the axes of the image in order to know where each circle goes, we use plt.gca() which means "Get Current Axis" and store the axis information into "ax". Then when we add the circle patches to the image, they will appear in the correct places. Finally, since there is more than one blob, we will create the circular patches and add them to the image one-by-one using a while loop. 

In [ ]:
ax = plt.gca()
ax.imshow(image,origin='lower')
cnt = 0
while cnt < len(blobs_log):
    c = plt.Circle((blobs_log[cnt][1], blobs_log[cnt][0]), blobs_log[cnt][2], color='white', linewidth=2, fill=False)
    ax.add_patch(c)
    cnt = cnt + 1
plt.show()

#Exercise 8: Blob Parameters

Play around with the input parameters in the following block of code, then run it to see how changing they will affect the number (more blobs fewer blobs, no blobs) and type of blobs detected (bigger, smaller, brighter, fainter). 

In [ ]:
blobs_log = blob_log(image, min_sigma = 2, max_sigma=9, num_sigma=8, threshold=.05)

### Play with the paramters above this line ###
blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)
ax = plt.gca()
ax.imshow(image,origin='lower')
cnt = 0
while cnt < len(blobs_log):
    c = plt.Circle((blobs_log[cnt][1], blobs_log[cnt][0]), blobs_log[cnt][2], color='white', linewidth=2, fill=False)
    ax.add_patch(c)
    cnt = cnt + 1
plt.show()

Write down your findings below, such as what parameters give you no blobs, how does changing the parameters affect the results, and anything else that you find interesting!

In [ ]:
# RECORD YOUR FINDINGS HERE

### Difference of Gaussian (DoG)
This algorithm is an approximation of the Laplacian of Gaussian, where rather than actually computing the "Laplacian" of the Gaussian that we mentioned above, DoG approximates this computation by taking the difference of two successively smoothed (blurred) images. Blobs are then detected as bright-on-dark spots in these difference images. This algorithm has the same disadvantage for larger blobs as the Laplacian of Gaussian algorithm.

To run this algorithm, we use the function "blob_dog" (https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_dog), which takes the same parameters as blob_log, except for **num_sigma**, because the algorithm needs to take the difference between successive smoothings so it figures out the number of times it needs to smooth the image. This function returns the same type of array as "blob_log" : (y-coord, x-coord, size). 

#Exercise 9: DoG 

Fill in the input parameters for the call to the blob_dog function and run the code. How is the result from this function different from the previous one?

In [ ]:
blobs_dog = blob_dog(image, min_sigma=2,max_sigma=9, threshold=0.05)
blobs_dog[:, 2] = blobs_dog[:, 2] * np.sqrt(2)

ax = plt.gca()
ax.imshow(image,origin='lower')
cnt = 0
while cnt < len(blobs_dog):
    c = plt.Circle((blobs_dog[cnt][1], blobs_dog[cnt][0]), blobs_dog[cnt][2], color='white', linewidth=2, fill=False)
    ax.add_patch(c)
    cnt = cnt + 1
plt.show()

### Determinant of Hessian
The final option that scikit-image provides is the Determinant of Hessian algorithm. This algorithm performs a different kind of mathematical operation on the smoothed images, it calculates the Hessian matrix of the image (but we won't go into that), and searches for maxima in this matrix. It it the fastest approach, but it is also less accurate and has trouble finding very small blobs. 

To try it out, we will run the function "blob_doh" (https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_doh), which takes as input the same parameters as the "blob_log" function. Let's try it out!

# Exercise 10: DoH
Fill in the input parameters for the call to the blob_doh function and run the code. How is the result from this function different from the previous one? How were the input parameters different from the previous algorithms?

In [ ]:
blobs_doh = blob_doh(image, min_sigma=2, max_sigma=9, num_sigma=8, threshold=.001)

ax = plt.gca()
ax.imshow(image,origin='lower')
cnt = 0
while cnt < len(blobs_doh):
    c = plt.Circle((blobs_doh[cnt][1], blobs_doh[cnt][0]), blobs_doh[cnt][2], color='white', linewidth=2, fill=False)
    ax.add_patch(c)
    cnt = cnt + 1
plt.show()

# Exercise 11: Blobs 2
How do the three algorithms compare to each other? Do their results agree with what you would intuitively call a blob? Do you trust one algorithm more than another? 


In [ ]:
# RECORD YOUR THOUGHTS HERE


---

># **Particle Tracking**

Now that we have learned several image processing techniques, we are ready to put them together in an application. In physics, there is an experiment called the Monopole Ion Trap that uses electric fields to trap particles as show below.

![alt text](https://imgur.com/1cBAcEc.gif)

The basic priciple is that the particle is attracted to and repeled by the rod at the top of the image. In the image above, we have a very stable, well behaved, trap in which the ion just goes back and forth between two locations. However, under certain curcumstances, the particle may start to behave erratically, as shown below.

![alt text](https://imgur.com/Z575ope.gif)

We would like to track the particle so that we can study its position and velocity.

Lets start by importing the two movies shown above into python. We have saved each movie, image by image, inside of folders on the computers. So we will import them as follows:

In [ ]:
# we import the operating system library OS to help us import many files at once
import os

folder_location = "./P1"
#the following line will navigate python to the correct foler. chdir stands for change directory.
os.chdir(folder_location)
#the following line return a list of file names in the folder
files = os.listdir()



Now is a good time to notice that we don't actually need the entire image to do this computation. The only thing we really need is the region the particle travels within, as shown below 

![alt text](https://imgur.com/oXLY4RF.gif)

This is called the Region of Interest (ROI) of an image so we will crop every single image as we import it. Lets do that by importing a test image and checking how we want to crop it:

# Exercise 11: Cropping 2

Find the correct copping location by chaning the slice location

In [ ]:
test_image = io.imread(files[0])
plt.imshow(test_image[210::], cmap = cm.gray)


Now that we know where we want to crop the images, we can do so automatically for all of the images in the folder

In [ ]:
# the following line creates an empty list that we will populate with the images as we import and crop them. 
#That is, ROI is a list of matrices, each one representing an image.
ROIs = []

# the following for-loop imports the images
for image in files:
    ROIs.append(io.imread(image)[210::])

# to make sure we are doing things correctly, lets see what the 17th image in the list is
plt.imshow(ROIs[16], cmap=cm.gray)

Looks good. Okay, now that we have all of the images imported, we want to run a blob finding algorithm that finds the position of the blob in each of the images. However, there are "specles" in the background, so make sure that your blob finding parameters are set correctly.

# Exercise 12: Particle Tracking


The goal is for you to use the examples we have provided above to write your own code to systematically find the particles in the ROI list. A general outline of the code is provided below, but if you need further help feel free to ask. 

general outline:
1. choose one of the images in the ROI list to test your parameters on
2. apply one of the blob finding techniques to the image (take a look at earlier examples if needed)
3. make a for-loop that applies this technique to all of the images in ROI and collects all of the outputs needed

In [ ]:
#student solution goes here

In [ ]:
#instructor solution

particles = []

for image in ROIs:
    blob = blob_log(image, min_sigma = 5, max_sigma=10, num_sigma=2, threshold=.1)
    particles.append(blob)
    
#print(particles)

We should now have a list of particle locations and sizes. Note that the size information is not important to this application, so we can get rid of it. In general, we find it easier to work with big complicated lists using numpy, so first we will convert the list into a numpy array, and then clean it up.

In [ ]:
#the following line converts the list into a numpy array
particles = np.array(particles)
#the following line cleans up array to make it look nicer
particles = particles[::,0]
#this shows us what they array looks like.
print(particles)

# Data Analysis

Great, using a ROI made this computation faster, and it made sure that the particle was easy to find. Unfortunately it also introduced a small offset error in the verticle position of the particle. This can be seen by taking a look at the picture below:

![alt text](https://imgur.com/aI3a3Ve.gif)

To correct for this error we simply have to add an offset to the y position of the particle list.

In [ ]:
for n, blob in enumerate(particles):
    particles[n] = np.add(blob, [210,0,0])
print(particles)

The final step is to turn these pixel locations into measurements of distance between the particle and the center of the rod. To do this, we will use the usual distance formula:

![alt text](https://i.imgur.com/xsRJGbR.png)

where (y_2, x_2) is the location of the center of the rod and (y_1, y_2) is the location of the particle. From previous measurements, the experimenters know that the center of the rod is approximately at 

y_2 = 83

x_2 = 137

so we can change all of the particle *location* data to particle *distance* data as follows:

In [ ]:
distances = []
for particle in particles:
    distance = np.sqrt((83 - particle[0])**2 + (137-particle[1])**2)
    distances.append(distance)
    
distances = np.array(distances)
#print(distances)

Great, so now we have the the distances of the particles from the center of the rod for each movie. The experimenters also know two important pieces of information. 

1. the camera was taking 2360 pictures per second (FPS), so the time between each image is 1/2360 seconds. 
2. the distance between each pixel 5.9 micron = .0059 millimeters (mm)

we can use this information to make a plot of the particle's distance as a function of time with proper units.

In [ ]:
time = np.linspace(0,len(distances)/2360, len(distances) )
distances = distances*.0059
plt.plot(time,distances)
plt.xlabel('time(s)')
plt.ylabel('distance(mm)')

## Congratulations!

You just did particle tracking. Now we'll quickly demonstrate how to find the velocity of the particle.

Recall that the definition of velocity is simply distance/time. Since we know that the time between pictures is 1/2360 seconds, all we have to do is calculate the distance the particle moved between each frame.

In [ ]:
velocities = []
for n in range(len(distances)):
    if n < (len(distances)-1):
        velocity = (distances[n+1] - distances[n])*2360
        velocities.append(velocity)
#print(velocities)

Sometime is it useful/interesting to see the velocity data in a "phase diagram", which is just a plot of the position vs the velocity:

In [ ]:
plt.scatter(distances[:-1:],velocities, )
plt.xlabel('distance(mm)')
plt.ylabel('velocity(mm/s)')

As you can see, the stable particle makes a circle in these "phase diagrams". As you can try (below), the unstable particle will produce sometehing that looks like scribles instead. Phase diagrams are often used to quickly check the stability of a particle, without having to watch the full movie.